## Predicting Match Winners in the Premier League Using Scraped Data

In [ ]:
# Import Libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

#### Read in Data

In [ ]:
matches = pd.read_csv("prem_matches.csv", index_col=0)

#### Coding Safety Checks

In [ ]:
# round values
matches["round"].value_counts()

# delete comp and notes columns
del matches["comp"]
del matches["notes"]

# create datetime column and target as int
matches["date"] = pd.to_datetime(matches["date"])
matches["target"] = (matches["result"] == "W").astype("int")

# make sure codes are correct
matches["venue_code"] = matches["venue"].astype("category").cat.codes
matches["opp_code"] = matches["opponent"].astype("category").cat.codes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
matches["day_code"] = matches["date"].dt.dayofweek


#### Start Machine Learning

In [ ]:
# instantiate predictor
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

# train and test
train = matches[matches["date"] < '2023-09-01']
test = matches[matches["date"] > '2023-09-01']

# fit the model
predictors = ["venue_code", "opp_code", "hour", "day_code"]
rf.fit(train[predictors], train["target"])

#### Look at Error

In [ ]:
# check error
preds = rf.predict(test[predictors])
error = accuracy_score(test["target"], preds)

In [ ]:
# look at metric
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

#### Check Precision Score

In [ ]:
precision_score(test["target"], preds)

#### Create Grouped Matches

In [ ]:
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")

#### Create Rolling Average Function

In [ ]:
# create rolling averages and assign them
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

# create columns
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

#### Create Rolling Match Averages

In [ ]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

#### Create a Make Prediction Function

In [ ]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

#### Check Error

In [ ]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

#### Adjust Some Names

In [ ]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

combined["new_team"] = combined["team"].map(mapping)
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

#### Finally, Have Predictions

In [ ]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()